# Import Libraries and Download the Dataset

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


ds, info = tfds.load('tf_flowers',
              # take 80% for training, 10% for validation, and 10% for testing
              split=["train[:80%]", "train[80%:90%]", "train[90%:100%]"],
              as_supervised=True,
              with_info=True)
train_set, valid_set, test_set = ds

In [2]:
info.splits['train'].num_examples

In [3]:
class_names = info.features['label'].names
class_names

# Visualize the images

In [4]:
plt.figure(figsize=(10, 10))
i = 0
for image, label in train_set.take(9):
    plt.subplot(3, 3, i + 1)
    image = tf.image.resize(image, (224, 224))
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(class_names[label])
    plt.axis("off")
    i += 1

# Prepare the Dataset

In [5]:
# declare some variables
batch_size = 32
img_height = 224
img_width = 224
AUTOTUNE = tf.data.AUTOTUNE

# We will use this layer to standardize the pixel values
rescaling_layer = layers.Rescaling(1./255)


def create_dataset(ds, shuffle=False):
    if shuffle:
        ds = ds.shuffle(1000)
    # resize the images
    ds = ds.map(
        lambda x, y: (tf.image.resize(x, (img_height, img_width)), y)
    )
    # standardize the pixel values to the [0 ,1] range
    ds = ds.map(lambda x, y: (rescaling_layer(x), y))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_set = create_dataset(train_set, shuffle=True)
valid_set = create_dataset(valid_set)
test_set = create_dataset(test_set)

# Extract the Features

In [6]:
from keras.applications.vgg16 import VGG16

img_size = (img_height, img_width, 3)
conv_base = VGG16(input_shape=(img_size),
                   include_top=False,
                   weights='imagenet')

In [7]:
conv_base.summary()

In [8]:
def extract_features(dataset):
    # here we will store the extracted features and their labels
    features = []
    labels = []

    total_batchs = tf.data.experimental.cardinality(dataset)
    current_batch = 1

    # loop over the dataset to get batches of images and their labels
    for images_batch, labels_batch in dataset:
        print("[INFO] processing batch {}/{}".format(current_batch, total_batchs))
        # extract the features using the predict method
        # the shape will be (32, 7, 7, 512)
        features_batch = conv_base.predict(images_batch)

        # store the current batch of features and labels in a list
        features.append(features_batch)
        labels.append(labels_batch)
        current_batch += 1

    features = np.vstack(features) # shape: (2936, 7, 7, 512)
    labels = np.hstack(labels)     # shape: (2936,)
    # flatten the features
    features = features.reshape(features.shape[0], 7 * 7 * 512)

    return features, labels


print('[INFO] extracting features from training dataset ...')
train_features, train_labels = extract_features(train_set)
print('[INFO] extracting features from validation dataset ...')
validation_features, validation_labels = extract_features(valid_set)

# Train a New FC Classifier on the Extracted Features

In [9]:
model = Sequential([
    layers.Dense(128, activation='relu', input_dim=7 * 7 * 512),
    layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_features, train_labels,
                    epochs=10,
                    validation_data=(validation_features, validation_labels))

# Visualize the Results

In [10]:
def plot_learning_curves():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label="Training Accuracy")
    plt.plot(val_acc, label="Validation Accuracy")
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(loss, label="Training Loss")
    plt.plot(val_loss, label="Validation Loss")
    plt.legend()
    plt.grid(True)

    plt.show()

plot_learning_curves()

# Transfer Learning with Data Augmentation

In [11]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", input_shape=(img_size)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
  ])

base_model = tf.keras.applications.VGG16(
        input_shape=img_size,
        include_top=False,
        weights="imagenet",
)
# freeze all layers of the base model
base_model.trainable = False

model = Sequential([
    data_augmentation,
    base_model,
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs=10
history = model.fit(
  train_set,
  validation_data=valid_set,
  epochs=epochs
)

In [12]:
plot_learning_curves()